## <center>COA Document Understanding: Task A

<center>by Clayton Cohn


This Colab notebook is a response to the Specright mini project code challenge's Task A. Task A was described as follows:

*In this mini project, you are challenged to build a data extraction solution that can efficiently extract a subset of COA fields from a single template from one supplier.*

Subset of COA fields:<br>
>● Product Description<br>
>● Lot Number<br>
>● Manufacture Date<br>
>● Test Name<br>
>● Test Results<br>
>● Test Specification

Deliverables:
>● (Required) A Jupyter Notebook with a readme explanation on how we can run your code.<br>
>● (Optional) A brief report in pdf format explaining the approaches, and results.


This solution is implemented using Google Colab. As such, the notebook must first be mounted to the user's Google Drive. This cell can (and should) be ignored if the user is running this script for his or her or their local file system. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Below is the constant ```HOME_PATH```. This must point to the folder that contains all of the single-supplier COA files. If there are any other files in the folder, the program will throw a runtime exception.

In [2]:
HOME_PATH = "drive/My Drive/Specright/Single_Suppliers_Docs/"

I utilized the [pdftotext](https://pypi.org/project/pdftotext/) PDF parser to convert the PDF files into workable text. To properly function, the package requires that the [poppler c++ interface](https://packages.debian.org/stretch/libpoppler-cpp-dev) be installed locally before pip is called to install the package. 

Since Colab runs in a linux enviornment, the ```apt-get install``` command was used to install the poppler interface prior to pip installing the pdftotext package.

In [3]:
!sudo apt-get install libpoppler-cpp-dev
!pip install pdftotext

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libpoppler-cpp-dev is already the newest version (0.62.0-2ubuntu2.12).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


Once the pdftotext library was imported, I created a ```parseFile``` helper function to parse a given PDF file and extract the necessary data. 

The function extracts all of the information as strings, except for the tests. The tests are extracted as a 2d list, where the first item (```tests[0]```) represents the tests' labels, and the remaining items represent the individual tests and their corresponding results and annotations.

The function is annotated inline.

In [4]:
import os
import pdftotext
import re

'''
Each of the test files had all of the pertinent information confined to
the first page except for one. As such, a check had to be
implemented to identify this edge condition. For new COA instances, the
below list will serve as a reference for which files need to have additional
pages read into the program.
'''
multi_page = ["coa-kaolin-clay-purple.pdf"]

def parseFile(dir, filename):
  with open(dir+filename, "rb") as f:
    pdf = pdftotext.PDF(f)

    # Each COA is separated into list of lines
    page = pdf[0].split('\n')

    # Check if our file has multiple pages
    if filename in multi_page:
      page += pdf[1].split('\n')

    # Variables created to hold the information extracted
    product_name = ""
    lot_numer = ""
    tests = []
    test_labels = []

    # These indices are used to identify where in the text the "tests" are
    begin_test = None
    test_label_ind = None
    end_test = None
    
    # Need to address COA instances with more than one page
    begin_test2 = None
    end_test2 = None

    # Text is read line by line
    for i in range(len(page)):

      # Whitespace is stripped for uniformity and readability
      page[i] = page[i].strip()

      '''
      Below is the logic used to extract the specific information.
      Manufacture date is not identified on the COAs, so it is not extracted.
      Lot number is extracted, but each COA has the same value for lot number.
      Parsing was based on strings that were common to all instances.
      '''
      if page[i].startswith("Product Name:"):
        product_name = page[i].split(":")[1].strip()
      elif page[i].startswith("Lot Number:"):
        lot_number = page[i].split(":")[1].strip()

        # Lot number truncated to NA for readability and identifiability
        if lot_number.lower().startswith("not available"):
          lot_number = "NA"

        '''
        The logic to identify the various tests required different
        "split words" due to a lack of uniformity.
        '''
      elif page[i].startswith("Expiration Date:") or \
        page[i].startswith("Shelf Life:"):
        test_label_ind = i + 1
        begin_test = i + 2
      elif page[i].startswith("The above data") or \
            page[i].startswith("All product characteristic") or \
            page[i].startswith("Disclaimer: This"):

        end_test = i

        # If we are only concerned with the first page, break
        if filename not in multi_page:
            break
      else:
        continue

    # Test labels were split on two or more spaces
    test_labels = re.split(r'\s{2,}', page[test_label_ind])

    '''
    These two edge cases needed special attention, as their tests' label
    structure differed from that of the others.
    '''
    if (filename == "coa-multi-herb-oily-skin.pdf"):
          test_labels = ["Analytical Tests", "Description", "Result"]
    if (filename == "coa-caprylyl-glycol-ethylhexylglycerin.pdf"):
          test_labels = ["Characteristics/Method", "Specificiations (min)", \
                         "Specifications (max)", "Result"]
          begin_test += 1

    # The first item in each COA's test list is the list of test labels 
    tests.append(test_labels)

    # Here we need to address the COAs with multiple pages
    if filename in multi_page:

      '''
      Multi-page instances must be addressed individually. This is not ideal,
      as any new COAs added to the dataset must be addressed with individual
      logic on a case-by-case basis. I would like to find an alternative means
      of doing this.
      '''
      if filename == "coa-kaolin-clay-purple.pdf":

        '''
        The below two lines are inefficient. I could have the search stop when
        I find the correct lines. I will circle back to this if I have 
        time, but for now I am okay with it, as the code only executes on one
        COA, so it will not be too computationally expensive.
        '''

        # These line identify start and end indexes for the tests on page 2
        begin_test2 = [num for num in range(len(page)) \
                       if page[num].startswith("Selenium")][0]
        end_test2 = [num for num in range(len(page)) \
                     if page[num].startswith("Total Coliforms")][0]

    '''
    Individual tests were split on two or more spaces and then appended
    to the 2d tests list.
    '''
    for i in range(begin_test, end_test):
      test = re.split(r'\s{2,}', page[i])
      if len(test) == len(test_labels):
        tests.append(test)

    # Again, if the COA length is > 1 page, we must extract tests from page 2
    if filename in multi_page:

      # Multi-page instances must be addressed individually
      if filename == "coa-kaolin-clay-purple.pdf":
        for i in range(begin_test2+2, end_test2 + 1):
          test = re.split(r'\s{2,}', page[i])
          if i == begin_test2+2:
            test = ["Selenium", test[0], test[1]]
          tests.append(test)

        '''
        This part is also inefficient and rather sloppy. There is an overlap
        that causes certain tests to be included twice, so the below code is
        a workaround to fix this. I will circle back to this as well if I have
        time; however, like the previous mention of inefficient code, this 
        inefficiency is confined to a single COA. My concern right now is 
        completing the assignment (both the code and the write-up), so I will 
        have to worry about the optimization later. 
        '''
        br = [i for i in range(len(tests)) \
              if tests[i][0].startswith("Phone")][0]
        for i in range(br, br+6):
          del tests[br]

    # The function returns the parsed PDF as a dictionary
    return {"product_name":product_name, "lot_number":lot_number, "tests":tests}

The below cell tests the ```parseFile``` function on one file before iterating through the entire directory. This cell was also used extensively for debugging and checking edge cases.

In [5]:
test_file = parseFile(HOME_PATH, "coa-kaolin-clay-purple.pdf")
print(test_file)

{'product_name': 'Kaolin Clay, Purple', 'lot_number': 'NA', 'tests': [['Property', 'Specification', 'Analysis'], ['Appearance', 'Powder', 'Pass'], ['Color', 'Prple', 'Pass'], ['Odor', 'Characteristic', 'Pass'], ['Granulometric distribution (medium', '15.00 µm MAX', '12.48 µm'], ['Organic Material', '2.00% MAX', '0.29%'], ['Loss for Dehydration', '4.00% MAX', '0.47%'], ['SiO2', '48-62%', '53.88%'], ['Fe2O3', '5.5-10.0%', '8.93%'], ['AI2O3', '15-26%', '21.59%'], ['Antimony', '<0.50 ppm', '0.15 ppm'], ['Arsenic', '<2 ppm', '<1.00 ppm'], ['Cadmium', '<0.10 ppm', '<0.01 ppm'], ['Chromium', '<28 ppm', '22.00 ppm'], ['Cobalt', '<2 ppm', '0.80 ppm'], ['Lanthanum', '<25 ppm', '17.50 ppm'], ['Lead', '<32 ppm', '28.00 ppm'], ['Mercury', '<1 ppm', '<0.01 ppm'], ['Nickel', '<3 ppm', '1.90 ppm'], ['Selenium', '<2 ppm', '<1.00 ppm'], ['Total Bacteria', '<100 CFU/g', 'Pass'], ['Mold & Yeast', '<10 CFU/g', 'Pass'], ['Staphylococcus aureus', 'Absent', 'Absent'], ['Pseudomonas aeruginosa', 'Absent', 'Abs

I created another helper function, ```getFiles```, to iterate through the directory containing all of the COAs and import, convert, and merge them into a single list.

In [6]:
def getFiles(d):
  files = []
  for fname in os.listdir(d):
    doc = parseFile(d, fname)
    files.append(doc)
  print("Returned {} files.".format(len(files)))
  return files

In [7]:
all_files = getFiles(HOME_PATH);

Returned 70 files.


This helper function was created to print all of the data in a readble manner. It as used solely for debuggind and visualiztion.

In [8]:
def printFiles(files):
  for d in files:
    for k, v in d.items():
      print("{}: {}".format(k, v))
    print()

It is recommended to keep the call to ```printFiles``` commented out when not in use in order to reduce the length of the notebook.

In [9]:
# printFiles(all_files)

List of COAs is then converted into Pandas DataFrame, where each COA is an instance. From here, we can apply an AI/ML algorithm to the COAs with minimal additional effort.

In [10]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(all_files)
df.head(100)

,product_name,lot_number,tests
0,Clay Mask Base,NA,"[[Specifications, Range, Results], [Appearance, Thick cream, Pass], [Color, Yellow to beige, Pass], [pH value, 5.0-6.5, Pass]]"
1,White Charcoal Powder,NA,"[[Assay, Specification, Results], [Appearance, Fine powder, Pass], [Color, Black, Pass], [Odor (triangle test), To match standard, Pass], [Particle Size (Mesh), 95% through 3000 mesh, Pass], [Ash %, <4.0, 3.3], [Moisture %, <12.0, 5.9], [Specific Density (g/ml), 0.45-0.55, 0.51], [Heavy Metals (ppm), <20.0, Pass], [Arsenic (ppm), <3.0, 0.31], [Lead (ppm), <10.0, 0.59], [Mercury (ppm), <1.0, 0.03], [Germanium (ppm), <1.0, 0.015], [Microbials, <100cfu/g, Pass]]"
2,Kaolin,NA,"[[Test, Reference, Analysis], [Identification, Conforms to USP, Meets requirements], [Description, Conforms to USP, Meets requirements], [Loss on ignition, Max. 15.%, 13.82%], [Acid solubles, Max. 2.0%, 0.64%], [Iron, Slt. Reddish tint, Slt. Tint], [Carbonate, No effervescence, Passes test], [Lead, Max. 10.0 ppm, Less than 10.0 ppm], [Microbial Limits, Conforms to USP, Passes test]]"
3,AHA Fruit Acids,NA,"[[Assay, Specification, Results], [Appearance, Pale Yellow, transparent, corresponds], [pH 1.2-2.5, 1.2-2.5, 1.8], [Specific Gravity @20°C, 1.245-1.280, 1.249 g/ml], [Lactic Acid, 29.0-35.5, 33.2 %], [Glycolic Acid, 14.0-16.5, 15.7 %], [Citric Acid, 2.2-3.1, 2.5 %], [Tartaric Acid, 0.9-1.5, 1.1 %], [Total aerobic mesophile plate count, Max 100 CFU/g, corresponds], [Yeast and Molds, Max 10 CFU/ml, corresponds]]"
4,Citric Acid,NA,"[[Analytical Tests, Specifications, Results], [Characteristics, White crystalline powder, Conforms], [Water solubility, Very, Conforms], [Ethanol solubility, Freely 96%, Conforms], [Ether solubility, Sparingly, Conforms], [Assay, 99.7-100.3%, 99.79%], [Water, 0.00-0.50%, 0.06%], [Calcium, <=30.0, Conforms], [Heavy metals as lead, <=5.0mg/kg, Conforms], [Iron, <=3.00, Conforms], [Arsenic, <=1.0mg/kg, Conforms], [Mercury, <=0.5, Conforms], [Chloride, <=5.0, Conforms], [Lead, <=0.5mg/kg, Conforms], [Oxalic acid, <=100, Conforms], [Sulphate (100mg/kg), <=100, Conforms], [Sulphated Ash, <=0.05, Conforms], [Residue on ignition (USP/FCC), <=0.05%, Conforms], [Bulk Density, 400-1300, Conforms], [Particles on mesh 25 (0.71mm), 0.00-10.00, Conforms], [Particles through 0,20mm, 0.00-10.00, 6.11], [Particles on mesh 0,63mm, 0.00-10.00, 2.20], [Particles <mesh 100 (0,150mm), 0.00-10.00, conforms]]"
5,Glycolic Acid,NA,"[[Analytical Tests, Specification, Analysis], [Appearance, Transparent, slightly yellowish liquid, pass], [Odor, Like burnt sugar, pass], [pH Value at 25 C, 0.5-2.0, 2.0], [Acid, Total, 70.0 Minimum, 70], [Specific Gravity, 1.27, Pass], [Water, %, 30.0 Maximum, 30], [Formic Acid, mg/kg, 100.0 Maximum, 100], [Color (Gardner), 1.0 Maximum, 1.0], [Sodium Chloride, %, 0.1-0.5, 0.3]]"
6,Lactic Acid,NA,"[[Analytical Tests, Specification, Analysis], [Appearance, Clear, water-white syrupy liquid, pass], [Odor, Characteristic, pass], [Color (Gardner), 50 Maximum, 17], [Assay, 88% Minimum, 88.1]]"
7,Salicylic Acid,NA,"[[Characteristic, Specifications, Results], [Chlorides, 0-0.0100, <0.0100], [Melting Range, 158-161°C, 160.4°C], [Identification, Pass, Pass], [Heavy Metals (as Pb), 0-20 ug/g, <20], [Loss on Drying, 0-0.500, 0.075], [Rsidue on ignition, 0-0.0500, 0.0176], [Sulphates, 0-0.020, <0.020], [Assay, 99.5-101.00, 99.98], [Related compounds, 0-0.200, 0.0578], [Phenol, 0-0.0100, <0.0010], [Other impurities, 0-0.0500, <0.0010], [4-Hydroxybenzoic Acid, 0-0.1000, 0.0292], [4-Hydroxyisophthalic Acid, 0-0.0500, 0.0286], [Sum of Impurities, 0-0.2000, 0.0578]]"
8,Shea Butter Glycerides,NA,"[[Parameter, Results], [Color (Gardner), 4.0], [Acid Value (mg KOH/g), 1.16], [Saponification Value (meq/kg), 163.2], [pH (1%IPA:H20), 6.8], [Moisture (%), 0.20]]"
9,Polyglyceryl Oleate,NA,"[[Property, Specification Range, Result], [Appearance @ 77 Deg C, Amber Liquid, Pass], [Color (Gardner), 9.0 max, 3.0], [Acid Value, 4.0 max, 2.5], [Saponification Va

Because the tests were stored as nested lists, I created a helper function to retrieve them from a given instance. The function takes two arguments, the instance's row and the DataFrame, and returns the nested list of tests for that particular instance as a 2d NumPy array (with ```tests[0]``` representing the labels for the specified COA's tests).

In [11]:
import numpy as np

def getTests(t, df):
  series = df.iloc[t]
  return np.array(series["tests"])

In [12]:
test = getTests(0, df)
print(test)

[['Specifications' 'Range' 'Results']
 ['Appearance' 'Thick cream' 'Pass']
 ['Color' 'Yellow to beige' 'Pass']
 ['pH value' '5.0-6.5' 'Pass']]
